In [1]:
ode_model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/min_system_e_coli.xml'
smoldyn_model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/smoldyn/MinE.txt'

In [2]:
from basico import * 
import smoldyn as sm
from smoldyn._smoldyn import MolecState

In [3]:
from process_bigraph import Composite, Process
from biosimulators_processes import CORE 


doc = {
         'smoldyn': {
             '_type': 'process',
             'address': f'local:smoldyn-process',
             'config': {
                 'model': {
                     'model_source': smoldyn_model_fp
                 }
             },
             'inputs': {
                 'species_counts': ['species_store'],
                 'molecules': ['molecules_store'],
             },
             'outputs': {
                 'species_counts': ['species_store'],
                 'molecules': ['molecules_store'],
             }
         },
         'emitter': {
             '_type': 'step',
             'address': 'local:ram-emitter',
             'config': {
                 'emit': {
                     'species_counts': 'tree[integer]',
                     'molecules': 'tree[string]'
                 }
             },
             'inputs': {
                 'species_counts': ['species_store'],
                 'molecules': ['molecules_store']
             }
         }
     }

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


In [12]:
smoldyn_model = sm.Simulation.fromFile(smoldyn_model_fp)

In [13]:
dir(smoldyn_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__todisp_text__',
 'accuracy',
 'addBidirectionalReaction',
 'addBox',
 'addCommand',
 'addCommandStr',
 'addCompartment',
 'addCompartmentLogic',
 'addCompartmentMolecules',
 'addCompartmentPoint',
 'addCompartmentSurface',
 'addCylinder',
 'addDisk',
 'addGraphics',
 'addHemisphere',
 'addLattice',
 'addLatticeMolecules',
 'addLatticePort',
 'addLatticeReaction',
 'addLatticeSpecies',
 'addMolList',
 'addMoleculePerBox',
 'addMolecules',
 'addOutputData',
 'addOutputFile',
 'addPanel',
 'addPanelNeighbor',
 'addPartition',
 'addPath2D',
 'addPort',
 'addPortMolecule',
 'addReaction',
 'addRectangle',
 'addSolutionMolecules',
 'addSpec

In [14]:
smoldyn_model.count()

{'compartment': 1,
 'dim': 3,
 'functions': 0,
 'species': 5,
 'surface': 1,
 'variables': 5}

In [15]:
species_count = smoldyn_model.count()['species']
species_names = []
for index in range(species_count):
    species_name = smoldyn_model.getSpeciesName(index)
    if 'empty' not in species_name.lower():
        species_names.append(species_name)
initial_species_counts = {
            spec_name: smoldyn_model.getMoleculeCount(spec_name, MolecState.all)
            for spec_name in species_names
        }

initial_species_counts

{'MinD_ATP': 4000, 'MinD_ADP': 0, 'MinE': 1400, 'MinDMinE': 0}

In [16]:
var_count = smoldyn_model.count()['variables']
for i in range(var_count):
    print(smoldyn_model.getReactionName(i, 0))

TypeError: getReactionName(): incompatible function arguments. The following argument types are supported:
    1. (self: smoldyn._smoldyn.Simulation, arg0: int, arg1: int, arg2: str) -> str

Invoked with: <smoldyn.smoldyn.Simulation object at 0x1685fc400>, 0, 0

In [17]:
import re


def parse_smoldyn_reactions(filepath):
    reactions = []
    definitions = {}
    with open(filepath, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.split('#')[0].strip()
            
            # parse definitions
            if line.startswith('define'):
                param = line.split('define ')
                param_data = param[1].split()
                definitions[param_data[0]] = param_data[1]
            
            # parse reactions and construct reaction data
            reaction = {}
            if line.startswith('reaction'):
                line_parts = line.split('reaction ')
                line_parts.pop(0)
                reaction_info = line_parts[0]
                pattern = r'^(\w+)\s+(.+->.+)\s+(\w+)$'
                match = re.match(pattern, reaction_info)
                if match:
                    name = match.group(1) 
                    rxn = match.group(2)  
                    rate_constant = match.group(3) 
                    reaction['name'] = name
                    reaction['reaction'] = rxn
                    reaction['rate_constant'] = {'name': rate_constant, 'value': definitions[rate_constant]}
                    reactions.append(reaction)
                
    return reactions


reactions = parse_smoldyn_reactions(smoldyn_model_fp)

In [18]:
reactions

[{'name': 'rxn1b',
  'reaction': 'MinD_ATP(fsoln) + MinD_ATP(front) -> MinD_ATP(front) + MinD_ATP(front)',
  'rate_constant': {'name': 'SIGMA_dD', 'value': '0.0015'}},
 {'name': 'rxn1c',
  'reaction': 'MinD_ATP(fsoln) + MinDMinE(front) -> MinD_ATP(front) + MinDMinE(front)',
  'rate_constant': {'name': 'SIGMA_dD', 'value': '0.0015'}},
 {'name': 'rxn2',
  'reaction': 'MinE(fsoln) + MinD_ATP(front) -> MinDMinE(front)',
  'rate_constant': {'name': 'SIGMA_E', 'value': '0.093'}},
 {'name': 'rxn3',
  'reaction': 'MinDMinE(front) -> MinD_ADP(fsoln) + MinE(fsoln)',
  'rate_constant': {'name': 'SIGMA_de', 'value': '0.7'}},
 {'name': 'rxn4',
  'reaction': 'MinD_ADP -> MinD_ATP',
  'rate_constant': {'name': 'SIGMA_D_D2T', 'value': '1'}}]

In [19]:
from basico import * 
import numpy as np


In [20]:
model = new_model(name='MinE')

In [21]:
for reaction in reactions:
    rxn_name = reaction['name']
    rxn = reaction['reaction']
    rate_constant_name = reaction['rate_constant']['name']
    rate_constant_value = reaction['rate_constant']['value']
    
    # add reaction to copasi
    add_reaction(rxn_name, rxn, model=model)
    set_reaction_parameters(f'({rxn_name}).k1', value=float(rate_constant_value))
    

In [22]:
get_reaction_parameters(model=model)

,value,reaction,type,mapped_to
name,,,,
(rxn1b).k1,0.0015,rxn1b,local,
(rxn1c).k1,0.0015,rxn1c,local,
(rxn2).k1,0.0930,rxn2,local,
(rxn3).k1,0.7000,rxn3,local,
(rxn4).k1,1.0000,rxn4,local,


In [23]:
get_species(model=model)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
MinD_ATP(front),compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_0,,0.0,MinD_ATP(front)
MinD_ATP(fsoln),compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_1,,0.0,MinD_ATP(fsoln)
MinDMinE(front),compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_2,,0.0,MinDMinE(front)
MinE(fsoln),compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_3,,0.0,MinE(fsoln)
MinD_ADP(fsoln),compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_4,,0.0,MinD_ADP(fsoln)
MinD_ADP,compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_5,,0.0,MinD_ADP
MinD_ATP,compartment,reactions,mol/l,1.0,6.022141e+23,,,NaN,NaN,0.0,0.0,Metabolite_6,,0.0,MinD_ATP


In [24]:
run_time_course(0, 10, 100, model=model, update_model=True)


In [29]:
get_reactions(model=model)

,scheme,flux,particle_flux,function,key,sbml_id,display_name,mapping
name,,,,,,,,
rxn1b,MinD_ATP(fsoln) + MinD_ATP(front) -> 2 * MinD_...,0.000297,1.789673e+20,Mass action (irreversible),Reaction_0,,(rxn1b),"{'k1': 0.0015, 'substrate': ['MinD_ATP(fsoln)'..."
rxn1c,MinD_ATP(fsoln) + MinDMinE(front) -> MinD_ATP(...,0.000101,6.067232e+19,Mass action (irreversible),Reaction_1,,(rxn1c),"{'k1': 0.0015, 'substrate': ['MinD_ATP(fsoln)'..."
rxn2,MinE(fsoln) + MinD_ATP(front) -> MinDMinE(front),0.036014,2.168808e+22,Mass action (irreversible),Reaction_2,,(rxn2),"{'k1': 0.093, 'substrate': ['MinE(fsoln)', 'Mi..."
rxn3,MinDMinE(front) -> MinD_ADP(fsoln) + MinE(fsoln),0.047565,2.864406e+22,Mass action (irreversible),Reaction_3,,(rxn3),"{'k1': 0.7, 'substrate': 'MinDMinE(front)'}"
rxn4,MinD_ADP -> MinD_ATP,0.000045,2.734023e+19,Mass action (irreversible),Reaction_4,,(rxn4),"{'k1': 1.0, 'substrate': 'MinD_ADP'}"


In [30]:
run_time_course(10, 20, 100, model=model, update_model=True)

,MinD_ATP(front),MinDMinE(front),MinD_ADP,MinD_ATP(fsoln),MinD_ADP(fsoln),MinE(fsoln),MinD_ATP
Time,,,,,,,
10.0,0.200433,0.067949,4.539952e-05,0.988469,2.743149,1.932051,1.999955
10.2,0.193431,0.065677,3.716997e-05,0.988390,2.752502,1.934323,1.999963
10.4,0.186665,0.063478,3.043219e-05,0.988315,2.761542,1.936522,1.999970
10.6,0.180129,0.061349,2.491577e-05,0.988242,2.770279,1.938651,1.999975
10.8,0.173815,0.059290,2.039930e-05,0.988172,2.778723,1.940710,1.999980
11.0,0.167717,0.057296,1.670153e-05,0.988104,2.786883,1.942704,1.999983
11.2,0.161826,0.055367,1.367405e-05,0.988038,2.794769,1.944633,1.999986
11.4,0.156137,0.053499,1.119537e-05,0.987975,2.802389,1.946501,1.999989
11.6,0.150642,0.051692,9.165989e-06,0.987914,2.809752,1.948308,1.999991


In [34]:
get_reactions('rxn1b', model=model).flux[0]

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_17254/1502911509.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



4.854262348216608e-05

In [4]:
fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'
doc = {
        'copasi': {
            '_type': 'process',
            'address': f'local:copasi',
            'config': {
                'model_file': fp
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store']
            },
            'outputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_derivatives': ['reaction_derivatives_store']
            }
        },
        'cobra': {
            '_type': 'process',
            'address': f'local:cobra',
            'config': {
                'model_file': fp
            },
            'inputs': {
                'reaction_derivatives': ['reaction_derivatives_store']
            },
            'outputs': {
                'fluxes': ['fluxes_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'floating_species_concentrations': 'tree[float]',
                    'reaction_derivatives': 'tree[float]',
                    'fluxes': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_derivatives': ['reaction_derivatives_store'],
                'fluxes': ['fluxes_store']
            }
        }
    }

In [5]:
c = Composite(config={'state': doc}, core=CORE)

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

In [6]:
c.run(20)

/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/biosimulators_processes/processes/copasi_process.py:471: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/biosimulators_processes/processes/copasi_process.py:480: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [7]:
c.gather_results()

{('emitter',): [{'time': 0.0,
   'species_concentrations': {'LacI protein': 0.0,
    'TetR protein': 0.0,
    'cI protein': 0.0,
    'LacI mRNA': 0.0,
    'TetR mRNA': 20.0,
    'cI mRNA': 0.0},
   'reaction_derivatives': {'degradation of LacI transcripts': 0.0,
    'degradation of TetR transcripts': 6.931471805599453,
    'degradation of CI transcripts': 0.0,
    'translation of LacI': 0.0,
    'translation of TetR': 138.62943611198907,
    'translation of CI': 0.0,
    'degradation of LacI': 0.0,
    'degradation of TetR': 6.931471805599453,
    'degradation of CI': 0.0,
    'transcription of LacI': 30.0,
    'transcription of TetR': 30.0,
    'transcription of CI': 30.0},
   'fluxes': {}},
  {'time': 1.0,
   'species_concentrations': {'LacI protein': 81.44046884923394,
    'TetR protein': 188.3821831030562,
    'cI protein': 42.64131125320374,
    'LacI mRNA': 19.90344166947679,
    'TetR mRNA': 50.61558010446934,
    'cI mRNA': 7.491001903157758},
   'reaction_derivatives': {'degra

In [9]:
import cobra
from cobra.io import read_sbml_model


mod = read_sbml_model('/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

In [10]:
sol = mod.optimize()

In [11]:
sol.fluxes

Reaction1     0.0
Reaction2     0.0
Reaction3     0.0
Reaction4     0.0
Reaction5     0.0
Reaction6     0.0
Reaction7     0.0
Reaction8     0.0
Reaction9     0.0
Reaction10    0.0
Reaction11    0.0
Reaction12    0.0
Name: fluxes, dtype: float64

In [14]:
for reaction in mod.reactions:
    print(reaction.id, reaction.name)

Reaction1 degradation of LacI transcripts
Reaction2 degradation of TetR transcripts
Reaction3 degradation of CI transcripts
Reaction4 translation of LacI
Reaction5 translation of TetR
Reaction6 translation of CI
Reaction7 degradation of LacI
Reaction8 degradation of TetR
Reaction9 degradation of CI
Reaction10 transcription of LacI
Reaction11 transcription of TetR
Reaction12 transcription of CI
